In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import re

from scipy.io import readsav

from astropy.time import Time
from astropy.time import TimeDelta

from sunpy import timeseries as ts
from sunpy.net import attrs as a
from sunpy.net import Fido
from sunpy.time import parse_time, find_time

import pandas as pd

import goesxrs_temp as gtem #from Ian's functions
from Onsets_temp import *

# Just setup plot fonts
plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
                         'font.sans-serif':"Arial",'mathtext.default':"regular", 'axes.linewidth' : 2})

In [ ]:
tsearch=a.Time("2015-01-01","2015-01-10")

## Choosing a Flare to analyse

##### First choose a time range (day) to look at. Returns all flares of all classes detected during that period.

### Downloading GOES Data

We want to download this flare data if we haven't got it downloaded already. So we search our directory first, then we download if it is not present. (Set up for GOES-15 2s data)

In [ ]:
data_dir = r"C:\Users\William\Documents\University\MastersProj\Data\Run_1" #data directory

In [ ]:
def data_download(fl_data):
    print("{0} with peak time at: {1}".format(fl_data["fl_goescls"], fl_data["event_peaktime"]))

    # # Want data for +/-10 mins of flare, using astopy time to do this
    # gtstart=Time(fl_data["event_starttime"],scale='utc')-TimeDelta(10*60,format='sec')
    # gtend=Time(fl_data["event_endtime"],scale='utc')+TimeDelta(10*60,format='sec')

    # # Search and get the XRS data
    # tflrange=a.Time(gtstart.iso,gtend.iso)
    # rg15 = Fido.search(tflrange, a.hek.EventType("FL"), a.hek.FL.GOESCls > "C1.0", a.hek.FRM.Name == "SSW Latest Events")
    # print(len(rg15))
    fg15 = Fido.fetch(fl_data, path=data_dir)

In [ ]:
res = Fido.search(tsearch, a.hek.EventType("FL"), a.hek.FL.GOESCls > "C1.0", a.hek.FRM.Name == "SSW Latest Events")
#res = Fido.search(tsearch,a.hek.EventType("FL"),a.hek.FL.GOESCls > "C1.0",a.hek.OBS.Observatory == "GOES")
# res = Fido.search(tsearch,a.hek.EventType("FL"),a.hek.OBS.Observatory == "GOES")
fullres = res["hek"]

# Reduce the table down to just the info we want
srch_res = fullres["event_starttime", "event_peaktime",
                        "event_endtime", "fl_goescls", "hpc_x", "hpc_y", "ar_noaanum"]
                        # More stuff that could be useful, Heliographic lat lon, and where from
                        # "event_endtime", "fl_goescls", "hpc_x", "hpc_y", "ar_noaanum", "hgc_x", "hgc_y", "frm_name"]


print(f"Total Number of flares in period: {len(srch_res)}")
print("")

print(srch_res)


Total Number of flares in period: 65

    event_starttime          event_peaktime     ...    hpc_y    ar_noaanum
----------------------- ----------------------- ... ----------- ----------
2014-12-31 23:02:00.000 2014-12-31 23:39:00.000 ...   -55.47726       2253
2015-01-01 04:57:00.000 2015-01-01 05:00:00.000 ...    -69.4254       2253
2015-01-01 07:22:00.000 2015-01-01 07:30:00.000 ...   -51.59418       2253
2015-01-01 19:03:00.000 2015-01-01 19:05:00.000 ...   -44.10774       2253
2015-01-02 19:35:00.000 2015-01-02 19:38:00.000 ... -52.8042552       2253
2015-01-02 19:47:00.000 2015-01-02 19:48:00.000 ...   -316.0818       2248
2015-01-03 01:13:00.000 2015-01-03 01:17:00.000 ...   -34.21584       2253
2015-01-03 05:52:00.000 2015-01-03 05:54:00.000 ...   -33.20418       2253
2015-01-03 05:52:00.000 2015-01-03 05:57:00.000 ...   -32.90022       2253
2015-01-03 06:01:00.000 2015-01-03 06:03:00.000 ...   -33.19254       2253
                    ...                     ... ...         ..

In [ ]:
start_t =[]
peak_t=[]
end_t=[]
goes_class=[]
hpc_x=[]
hpc_y=[]
ar_num=[]

for fl in srch_res:
    #fetching the date 'YYYYMMDD' in string format fron the start date put in earlier
    day_string = Time(fl["event_starttime"],scale='utc').datetime.date().strftime("%Y%m%d")
    fl_path = os.path.join(data_dir,f'*g15*{day_string}*.nc')

    #tries to load in the file, if it does not exist, it will download it.
    try:
        ts.TimeSeries(fl_path, concatenate=True)
        print("File Exists")
    except ValueError:
        print("File does not exist.... Downloading file....")
        data_download(fl)
        fg15 = Fido.fetch(fl, path=data_dir)
        

File does not exist.... Downloading file....
C1.2 with peak time at: 2014-12-31 23:39:00.000


ValueError: Query result has an unrecognized type: <class 'sunpy.net.hek.hek.HEKRow'> Allowed types are QueryResponseRow, QueryResponseTable or UnifiedResponse.